```
Licensed to the Apache Software Foundation (ASF) under one
or more contributor license agreements.  See the NOTICE file
distributed with this work for additional information
regarding copyright ownership.  The ASF licenses this file
to you under the Apache License, Version 2.0 (the
"License"); you may not use this file except in compliance
with the License.  You may obtain a copy of the License at
  http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing,
software distributed under the License is distributed on an
"AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
KIND, either express or implied.  See the License for the
specific language governing permissions and limitations
under the License.
```

In [1]:
# EXECUTAR NO TERMINAL
# pip install pandas
# pip install apache-sedona
# COPIAR TIF PARA PASTA RASTER/BIG
# EXECUTAR FORBIGRASTER para dividir a Imagem em Imagens menores

## TODO - ENCONTRAR FORMA DE COPIAR DIRETO PARA O HADOOP PELO USUÀRIO (PARA FAZER PELO JUPYTER OLHAR ANOTACAO NO FIM DO ARQUIVO ForBigRaster)
# sudo docker exec -it hadoop bash
# hadoop fs -copyFromLocal /opt/workspace/raster/* /

In [2]:
from IPython.display import display, HTML
from pyspark.sql import SparkSession
from pyspark import StorageLevel
import pandas as pd
from pyspark.sql.types import StructType, StructField,StringType, LongType, IntegerType, DoubleType, ArrayType
from pyspark.sql.functions import regexp_replace
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from pyspark.sql.functions import col, split, expr
from pyspark.sql.functions import udf, lit
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from pyspark.sql.functions import col, split, expr
from pyspark.sql.functions import udf, lit

In [3]:
spark = SparkSession.\
    builder.\
    appName("Demo-app").\
    enableHiveSupport().\
    master("local[*]").\
    master("spark://spark-master:7077").\
    config("spark.executor.memory", "15G").\
    config("spark.driver.maxResultSize", "15G").\
    config("spark.serializer", KryoSerializer.getName).\
    config("spark.kryo.registrator", SedonaKryoRegistrator.getName).\
    config("spark.jars.packages", "org.apache.sedona:sedona-python-adapter-3.0_2.12:1.1.0-incubating,org.datasyslab:geotools-wrapper:1.1.0-25.2") .\
    getOrCreate()
#     config("spark.rpc.message.maxSize", 2047).\
# rdd = spark.sparkContext.parallelize(range(1000))
# rdd.takeSample(False, 5)

SedonaRegistrator.registerAll(spark)
sc = spark.sparkContext

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.sedona#sedona-python-adapter-3.0_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-158e8878-0532-4bc3-b6b9-016f34becad3;1.0
	confs: [default]
	found org.apache.sedona#sedona-python-adapter-3.0_2.12;1.1.0-incubating in central
	found org.locationtech.jts#jts-core;1.18.0 in central
	found org.wololo#jts2geojson;0.16.1 in central
	found com.fasterxml.jackson.core#jackson-databind;2.12.2 in central
	found com.fasterxml.jackson.core#jackson-annotations;2.12.2 in central
	found com.fasterxml.jackson.core#jackson-core;2.12.2 in central
	found org.apache.sedona#sedona-core-3.0_2.12;1.1.0-incubating in central
	found org.apache.sedona#sedona-sql

In [4]:
# Path to directory of geotiff images 
DATA_DIR = "hdfs://776faf4d6a1e:8020/tmp/"
df = spark.read.format("geotiff").option("dropInvalid",True).load(DATA_DIR)

In [5]:
df.cache()
df.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- wkt: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nBands: integer (nullable = true)
 |    |-- data: array (nullable = true)
 |    |    |-- element: double (containsNull = true)



In [6]:
df.is_cached

True

In [ ]:
# Java Heap Out Of Memory  => Ir nas máquinas e aumentar o export _JAVA_OPTIONS="-Xmx15g"
# Java lang Assertion Error image is too large =>
df = df.selectExpr("image.origin as origin","ST_GeomFromWkt(image.wkt) as Geom", "image.height as height", "image.width as width", "image.data as data", "image.nBands as bands").cache()
df.show(5)
# df.count()

In [ ]:
# ,"RS_GetBand(data, 2,bands) as Band2","RS_GetBand(data, 3,bands) as Band3", "RS_GetBand(data, 4,bands) as Band4"
df = df.selectExpr("Geom","RS_GetBand(data, 1,bands) as Band1","RS_GetBand(data, 2,bands) as Band2","RS_GetBand(data, 3,bands) as Band3", "RS_GetBand(data, 4,bands) as Band4").cache()
df.createOrReplaceTempView("allbands")
df.show(5)

In [ ]:
# spark.catalog.cacheTable('df')
# spark.catalog.isCached(tableName='df')

In [ ]:
NomalizedDifference = df.selectExpr("RS_NormalizedDifference(Band1, Band2) as normDiff").cache()
NomalizedDifference.show(5)

In [ ]:
meanDF = df.selectExpr("RS_Mean(Band1) as mean").cache()
meanDF.show(5)

In [ ]:
modeDF = df.selectExpr("RS_Mode(Band1) as mode").cache()
modeDF.show(5)

In [ ]:
greaterthanDF = spark.sql("Select RS_GreaterThan(Band1,1000.0) as greaterthan from allbands").cache()
greaterthanDF.show()

In [ ]:
greaterthanEqualDF = spark.sql("Select RS_GreaterThanEqual(Band1,360.0) as greaterthanEqual from allbands").cache()
greaterthanEqualDF.show()

In [ ]:
lessthanDF = spark.sql("Select RS_LessThan(Band1,1000.0) as lessthan from allbands").cache()
lessthanDF.show()

In [ ]:
lessthanEqualDF = spark.sql("Select RS_LessThanEqual(Band1,2890.0) as lessthanequal from allbands").cache()
lessthanEqualDF.show()

In [ ]:
sumDF = df.selectExpr("RS_AddBands(Band1, Band2) as sumOfBand").cache()
sumDF.show(5)

In [ ]:
subtractDF = df.selectExpr("RS_SubtractBands(Band1, Band2) as diffOfBand").cache()
subtractDF.show(5)

In [ ]:
multiplyDF = df.selectExpr("RS_MultiplyBands(Band1, Band2) as productOfBand").cache()
multiplyDF.show(5)

In [ ]:
divideDF = df.selectExpr("RS_DivideBands(Band1, Band2) as divisionOfBand").cache()
divideDF.show(5)

In [ ]:
mulfacDF = df.selectExpr("RS_MultiplyFactor(Band2, 2) as target").cache()
mulfacDF.show(5)

In [ ]:
bitwiseAND = df.selectExpr("RS_BitwiseAND(Band1, Band2) as AND").cache()
bitwiseAND.show(5)

In [ ]:
bitwiseOR = df.selectExpr("RS_BitwiseOR(Band1, Band2) as OR").cache()
bitwiseOR.show(5)

In [ ]:
countDF = df.selectExpr("RS_Count(RS_GreaterThan(Band1,1000.0), 1.0) as count").cache()
countDF.show(5)

In [ ]:
moduloDF = df.selectExpr("RS_Modulo(Band1, 21.0) as modulo ").cache()
moduloDF.show(5)

In [ ]:
rootDF = df.selectExpr("RS_SquareRoot(Band1) as root").cache()
rootDF.show(5)

In [ ]:
logDiff = df.selectExpr("RS_LogicalDifference(Band1, Band2) as loggDifference").cache()
logDiff.show(5)

In [ ]:
logOver = df.selectExpr("RS_LogicalOver(Band3, Band2) as logicalOver").cache()
logOver.show(5)

In [ ]:
df = spark.read.format("geotiff").option("dropInvalid",True).load(DATA_DIR)
df = df.selectExpr("image.origin as origin","ST_GeomFromWkt(image.wkt) as Geom", "image.height as height", "image.width as width", "image.data as data", "image.nBands as bands").cache()

df = df.selectExpr("RS_GetBand(data,1,bands) as targetband", "height", "width", "bands", "Geom")
df_base64 = df.selectExpr("Geom", "RS_Base64(height,width,RS_Normalize(targetBand), RS_Array(height*width,0.0), RS_Array(height*width, 0.0)) as red","RS_Base64(height,width,RS_Array(height*width, 0.0), RS_Normalize(targetBand), RS_Array(height*width, 0.0)) as green", "RS_Base64(height,width,RS_Array(height*width, 0.0),  RS_Array(height*width, 0.0), RS_Normalize(targetBand)) as blue","RS_Base64(height,width,RS_Normalize(targetBand), RS_Normalize(targetBand),RS_Normalize(targetBand)) as RGB" ).cache()
df_HTML = df_base64.selectExpr("Geom","RS_HTML(red) as RedBand","RS_HTML(blue) as BlueBand","RS_HTML(green) as GreenBand", "RS_HTML(RGB) as CombinedBand").cache()
df_HTML.show(5)

In [ ]:
display(HTML(df_HTML.limit(2).toPandas().to_html(escape=False)))

In [ ]:

def SumOfValues(band):
    total = 0.0
    for num in band:
        if num>1000.0:
            total+=1
    return total
    
calculateSum = udf(SumOfValues, DoubleType())
spark.udf.register("RS_Sum", calculateSum)

sumDF = df.selectExpr("RS_Sum(targetband) as sum").cache()
sumDF.show()

In [ ]:
def generatemask(band, width,height):
    for (i,val) in enumerate(band):
        if (i%width>=12 and i%width<26) and (i%height>=12 and i%height<26):
            band[i] = 255.0
        else:
            band[i] = 0.0
    return band

maskValues = udf(generatemask, ArrayType(DoubleType()))
spark.udf.register("RS_MaskValues", maskValues)


df_base64 = df.selectExpr("Geom", "RS_Base64(height,width,RS_Normalize(targetband), RS_Array(height*width,0.0), RS_Array(height*width, 0.0), RS_MaskValues(targetband,width,height)) as region" ).cache()
df_HTML = df_base64.selectExpr("Geom","RS_HTML(region) as selectedregion").cache()
display(HTML(df_HTML.limit(2).toPandas().to_html(escape=False)))

In [ ]:
spark.stop()